# Is there any place for a new Chinese restaurant in Toronto?

## Part 1: Problem Description

The idea of this study is to help people planning to open a new Chinese restaurant in Toronto. To chose the right location by providing data about the income and population of each neighborhood as well as the competitors already present on the same spot. It is widely unknown that Chinese restaurants are usually quite cheap, which is an idea choice for middle or low income residents. For examples, in New york or Los Angles, there are huge China town, where the compitition is usually very high.

In this project, I will try to clusters the different restaurants in Toronto city, and see how it is possible to start a new Chinese restaurant.


# Part 2, Data

The data we will use is comments from "Foursqure" and API from website. By giving it with Geolocation such as longitude and latitude, it is possible to see some comments from previous users. The most useful information, that I will use are the density of the Chinese resutaurant around there and the scores that people like it or not.

### Some Notes about "Foursquare": <https://foursquare.com/>
##### Foursquare is a local search-and-discovery service mobile app which provides search results for its users (Wikipedia).
##### Founded: New York City, New York, U.S
##### Users: 60 million
##### Date launched: March 11, 2009
##### Employees: Over 200
##### Founders: Dennis Crowley, Naveen Selvadurai
##### Owner: Foursquare Labs, Inc.

# Part 3. Data acquisation and analysis

In [18]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [4]:
#loading the postcode of Toronto
df_toronto = pd.read_csv('toroto_post.csv')
df_toronto.head()

,Unnamed: 0,Postcode,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,4,M7A,Queen's Park,Not assigned,43.662301,-79.389494


In [14]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [22]:
# create map of Toronto using latitude and longitude values
latitude=43.753259
longitude=-79.329656

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Which Borough has the highest average salary

In [23]:
# Toronto Open Data Catalogue - Neighbourhood Profiles 2016 (CSV)
# https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#8c732154-5012-9afe-d0cd-ba3ffc813d5a

csv_path='https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_neighbourhood_profiles.csv'
df = pd.read_csv(csv_path,encoding='latin1')
print('Data loaded')

Data loaded


In [24]:
df.head()

,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,42,34,76,52,49,39,112,127,122,24,69,108,41,57,30,71,109,96,133,75,120,33,123,92,66,59,47,126,93,26,83,62,9,138,5,32,11,13,44,102,101,25,65,140,53,88,87,134,48,8,21,22,106,125,14,90,110,124,78,6,15,114,117,38,105,103,56,84,19,132,29,12,130,17,135,73,115,2,99,104,18,50,36,82,68,74,121,107,54,58,80,45,23,67,46,10,72,4,111,86,98,131,89,28,139,85,70,40,116,16,118,61,63,3,55,81,79,43,77,136,1,35,113,91,119,51,37,7,137,64,60,94,100,97,27,31
1,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,No Designation,NIA,No Designation,No Designation,No Designation,NIA,NIA,Emerging Neighbourhood,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,NIA,NIA,No Designation,NIA,No Designation,No Designation,NIA,NIA,No Designat

In [25]:
Neighbourhoods = list(df.columns.values)
dfToronto = pd.DataFrame(index=Neighbourhoods, columns=["Population_2016","Income_2016"])
dfToronto.head()

,Population_2016,Income_2016
Category,NaN,NaN
Topic,NaN,NaN
Data Source,NaN,NaN
Characteristic,NaN,NaN
City of Toronto,NaN,NaN


In [8]:
# Population_2016 = Population, 2016
# Income_2016 = Total income: Average amount ($)


for index, row in dfToronto.iterrows():
    dfToronto.at[index, 'Population_2016'] = df[index][2]
    dfToronto.at[index, 'Income_2016'] = df[index][2264]
    

dfToronto.sort_values('Income_2016').head()

,Population_2016,Income_2016
St.Andrew-Windfields,"17,812","100,516"
Edenbridge-Humber Valley,"15,535","101,551"
Lawrence Park North,"14,607","111,730"
Annex,"30,526","112,766"
Yonge-St.Clair,"12,528","114,174"


In the above, we see the richeset neighbors that is in toronto, which may be a good choice for a new Chinese restaurant.

# Clustering the Borough into different areas with K-Means methods

#### Now, let's get the top 100 venues that are in St.Andrew_Windfields within a radius of 500 meters.

In [27]:
CLIENT_ID = 'TPW5BLJTMYEPP2CHDNHGY3HWXG02MKEHP13ECPYEWYRAQHJD' # your Foursquare ID
CLIENT_SECRET = 'XT0USZSBJLGFZ0XVTEEF1Q5KNSC5LZEM33O0DNUY4WQ4QJOV' # your Foursquare Secret
VERSION = '20190716'
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TPW5BLJTMYEPP2CHDNHGY3HWXG02MKEHP13ECPYEWYRAQHJD
CLIENT_SECRET:XT0USZSBJLGFZ0XVTEEF1Q5KNSC5LZEM33O0DNUY4WQ4QJOV


In [29]:
search_query = ''
radius = 500
limit=100
latitude=neighborhood_latitude=43.6419024324
longitude=neighborhood_longitude=-79.384248463
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=TPW5BLJTMYEPP2CHDNHGY3HWXG02MKEHP13ECPYEWYRAQHJD&client_secret=XT0USZSBJLGFZ0XVTEEF1Q5KNSC5LZEM33O0DNUY4WQ4QJOV&v=20190716&ll=43.6419024324,-79.384248463&radius=500&limit=30'

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d366dc849b14d0023110061'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Entertainment District',
  'headerFullLocation': 'Entertainment District, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 101,
  'suggestedBounds': {'ne': {'lat': 43.646402436900004,
    'lng': -79.37804174020927},
   'sw': {'lat': 43.6374024279, 'lng': -79.39045518579074}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53357710498e20817350cfb4',
       'name': 'Delta Hotels by Marriott Toronto',
       'location': {'address': '75 Lower Simcoe Street',
        'lat': 43.6428819,
        'lng': -79.3839491,
        'labeledLatLngs': [{'label':

In [31]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Delta Hotels by Marriott Toronto,Hotel,43.642882,-79.383949
1,Roundhouse Park,Park,43.641745,-79.384279
2,iQ Food Co,Salad Place,43.642851,-79.382081
3,Ripley's Aquarium of Canada,Aquarium,43.642104,-79.386252
4,Steam Whistle Brewing,Brewery,43.641752,-79.387089


In [34]:
nearby_venues

,name,categories,lat,lng
0,Delta Hotels by Marriott Toronto,Hotel,43.642882,-79.383949
1,Roundhouse Park,Park,43.641745,-79.384279
2,iQ Food Co,Salad Place,43.642851,-79.382081
3,Ripley's Aquarium of Canada,Aquarium,43.642104,-79.386252
4,Steam Whistle Brewing,Brewery,43.641752,-79.387089
5,Sky Pod,Scenic Lookout,43.642561,-79.387038
6,Ray Bay,Aquarium,43.642323,-79.385930
7,Aroma Espresso Bar,Café,43.642321,-79.383749
8,SOCO Kitchen+Bar,Bistro,43.642804,-79.383921
9,CN Tower,Monument / Landmark,43.642536,-79.387182


# From the category data, there seems to be no Chinese restaurant, which could be a very good opportunity to start up a restaurant there. 